In [1]:
import pandas as pd

In [2]:
df_long = pd.read_csv("../data/processed/crimes_against_women_long.csv")

## Aggregate yearly crime counts per district

In [3]:
district_features = (
    df_long
    .groupby(['STATE/UT', 'DISTRICT','Crime_Type'])['Crime_Count']
    .sum()
    .reset_index()
)

In [4]:
district_pivot = district_features.pivot_table(
    index=['STATE/UT', 'DISTRICT'],
    columns='Crime_Type',
    values='Crime_Count',
    fill_value=0
)


In [5]:
district_pivot.head()

Crime_Type                     Assault on women with intent to outrage her modesty  \
STATE/UT      DISTRICT                                                               
A & N ISLANDS A and N ISLANDS                                               21.0     
              ANDAMAN                                                      168.0     
              CAR                                                            1.0     
              NICOBAR                                                        8.0     
              NORTH                                                          2.0     

Crime_Type                     Cruelty by Husband or his Relatives  \
STATE/UT      DISTRICT                                               
A & N ISLANDS A and N ISLANDS                                 18.0   
              ANDAMAN                                         97.0   
              CAR                                              0.0   
              NICOBAR                                          1.0   
              NORTH                                            2.0   

Crime_Type                     Dowry Deaths  Importation of Girls  \
STATE/UT      DISTRICT                                              
A & N ISLANDS A and N ISLANDS           1.0                   0.0   
              ANDAMAN                   3.0                   0.0   
              CAR                       0.0                   0.0   
              NICOBAR                   0.0                   0.0   
              NORTH                     0.0                   0.0   

Crime_Type                     Insult to modesty of Women  \
STATE/UT      DISTRICT                                      
A & N ISLANDS A and N ISLANDS                         2.0   
              ANDAMAN                                37.0   
              CAR                                     0.0   
              NICOBAR                                 0.0   
              NORTH                                   1.0   

Crime_Type                     Kidnapping and Abduction  Rape  
STATE/UT      DISTRICT                                         
A & N ISLANDS A and N ISLANDS                       9.0   3.0  
              ANDAMAN                              61.0  92.0  
              CAR                                   1.0   0.0  
              NICOBAR                               0.0   2.0  
              NORTH                                 0.0   4.0

In [6]:
district_pivot.shape

(2518, 7)

## Scale features

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(district_pivot)

## Apply clustering (KMeans)

In [11]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5, random_state=42)
clusters = kmeans.fit_predict(X_scaled)

district_pivot['Cluster'] = clusters


## Inspect cluster profiles

In [12]:
cluster_summary = (
    district_pivot
    .groupby('Cluster')
    .mean()
    .round(1)
)

cluster_summary

Crime_Type  Assault on women with intent to outrage her modesty  \
Cluster                                                           
0                                                       298.9     
1                                                     44964.0     
2                                                      3895.3     
3                                                     29698.0     
4                                                     14298.4     

Crime_Type  Cruelty by Husband or his Relatives  Dowry Deaths  \
Cluster                                                         
0                                         540.4          48.0   
1                                       94021.6        5947.6   
2                                       11564.7        5778.7   
3                                       77617.0       23824.0   
4                                       21659.1        1781.5   

Crime_Type  Importation of Girls  Insult to modesty of Women  \
Cluster                                                        
0                            0.3                        68.5   
1                           39.8                     11893.2   
2                          265.3                       168.3   
3                            3.0                     26511.0   
4                            6.9                      2530.9   

Crime_Type  Kidnapping and Abduction     Rape  
Cluster                                        
0                              185.8    151.2  
1                            16295.2  20827.4  
2                             8368.0   7555.0  
3                            47590.0  19058.0  
4                             9174.5   6854.6

## Save ML artifacts

In [14]:
district_pivot.to_csv("../data/processed/district_crime_clusters.csv")

In [15]:
district_pivot.shape

(2518, 8)

In [16]:
cluster_summary

Crime_Type  Assault on women with intent to outrage her modesty  \
Cluster                                                           
0                                                       298.9     
1                                                     44964.0     
2                                                      3895.3     
3                                                     29698.0     
4                                                     14298.4     

Crime_Type  Cruelty by Husband or his Relatives  Dowry Deaths  \
Cluster                                                         
0                                         540.4          48.0   
1                                       94021.6        5947.6   
2                                       11564.7        5778.7   
3                                       77617.0       23824.0   
4                                       21659.1        1781.5   

Crime_Type  Importation of Girls  Insult to modesty of Women  \
Cluster                                                        
0                            0.3                        68.5   
1                           39.8                     11893.2   
2                          265.3                       168.3   
3                            3.0                     26511.0   
4                            6.9                      2530.9   

Crime_Type  Kidnapping and Abduction     Rape  
Cluster                                        
0                              185.8    151.2  
1                            16295.2  20827.4  
2                             8368.0   7555.0  
3                            47590.0  19058.0  
4                             9174.5   6854.6